In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

2023-04-29 11:09:20.253856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 11:09:20.470327: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-29 11:09:20.477117: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-29 11:09:20.477149: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

# CNN

## Chargement des Données

In [2]:
# Données d'entrainement
train_data_complete = pd.read_csv("../data/allocine_genres_train.csv", sep=",")
train_data = train_data_complete[["titre", "synopsis", "genre"]]

# Données de test/validation
test_data_complete = pd.read_csv("../data/allocine_genres_test.csv", sep=",")
test_data = test_data_complete[["titre", "synopsis", "genre"]]

In [3]:
X = train_data.drop('genre', axis=1)
y = train_data['genre']

# Initialize the RandomOverSampler
ros = RandomOverSampler()
#ros = RandomUnderSampler()


# Perform oversampling
X_resampled, y_resampled = ros.fit_resample(X, y)

# Convert the resampled data back to a DataFrame
train_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='genre')], axis=1)

Lister les classes et leur associer un identifiant unique. (Utile pour le plongement des mots et pour l'entraînement du CNN).

In [4]:
# Liste des genres
genre_name = sorted(train_data.genre.unique().flatten())
print("Genres:", genre_name)
print("Nombre d'exemplaires:", len(train_data))

# Identifiant unique par genre
genre2id = {genre_name[i]:i for i in range(len(genre_name))}
id2genre = {i:genre_name[i] for i in range(len(genre_name))}

Genres: ['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction']
Nombre d'exemplaires: 4509


Remplacer les genres par la valeur numérique associée.

In [5]:
train_data = train_data.replace({"genre": genre2id})
train_data.head()

,titre,synopsis,genre
0,Le Crime de l' Orient - Express,"En visite à Istanbul , le célèbre détective be...",6
1,12 hommes en colère,Un jeune homme d' origine modeste est accusé d...,3
2,Après moi le bonheur,"Lorsque Marie-Laure , mère de quatre jeunes en...",3
3,Les Lumières de la ville,Un vagabond s’ éprend d’ une belle et jeune ve...,7
4,Les Chemins de la dignité,"L' histoire vraie de Carl Brashear , premier A...",0


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data[["titre", "synopsis"]],
                                                    train_data[["genre"]],
                                                    test_size=0.001,
                                                    random_state=12, # Random seed for shuffle
                                                    shuffle=True)

On combine le titre et le synopsis pour pouvoir les vectoriser par la suite.

In [7]:
X_train_titre = X_train.titre
X_train = X_train_titre + " " + X_train.synopsis

In [8]:
X_train.head()

841     Tu mourras à 20 ans Soudan , province d’ Aljaz...
2949    Leto Leningrad . Un été du début des années 80...
868     Fleur de Tonnerre En 1800 , la Bretagne est à ...
1284    Les Profs 2 Les pires Profs de France débarque...
1267    Le Char et l' olivier , une autre histoire de ...
dtype: object

In [9]:
y_train.head()

,genre
841,3
2949,0
868,3
1284,1
1267,2


In [10]:
print(X_train.shape)
print(y_train.shape)

(4504,)
(4504, 1)


## Indexation du Vocabulaire

In [11]:
def get_vectorizer(documents, max_voc_size=8000, max_seq_length= 200, batch_size=64):
	vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_voc_size, output_sequence_length=max_seq_length)
	# Création du jeu de données à partir de X_train et constitution de lots de 128 instances
	text_ds = tf.data.Dataset.from_tensor_slices(documents).batch(batch_size)
	# Création du vocabulaire à partir des données d'entrée
	vectorizer.adapt(text_ds)
	return vectorizer

In [12]:
keras_vectorizer = get_vectorizer(X_train)

2023-04-29 11:09:26.485905: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-29 11:09:26.485994: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-29 11:09:26.486078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fabien): /proc/driver/nvidia/version does not exist
2023-04-29 11:09:26.486880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
voc = keras_vectorizer.get_vocabulary()
print(len(voc))

8000


In [14]:
voc[:10]

['', '[UNK]', 'de', 'la', 'et', 'le', 'à', 'un', 'les', 'une']

In [15]:
word_index = dict(zip(voc, range(len(voc))))
print(word_index)

{'': 0, '[UNK]': 1, 'de': 2, 'la': 3, 'et': 4, 'le': 5, 'à': 6, 'un': 7, 'les': 8, 'une': 9, 'l': 10, 'des': 11, 'est': 12, 'en': 13, 'il': 14, 'son': 15, 'dans': 16, 'd': 17, 'pour': 18, 'qui': 19, 'du': 20, 'se': 21, 'l’': 22, 'sa': 23, 'd’': 24, 'au': 25, 'a': 26, 'que': 27, 'par': 28, 'plus': 29, 'elle': 30, 'ses': 31, 'sur': 32, 'avec': 33, 'mais': 34, 'leur': 35, 'vie': 36, 'ce': 37, 's': 38, 'ils': 39, 'ne': 40, 'qu': 41, 'lui': 42, 'ans': 43, 'jeune': 44, 'pas': 45, 'tout': 46, 'va': 47, 'deux': 48, 'alors': 49, 'qu’': 50, '…': 51, 'sont': 52, 's’': 53, 'femme': 54, 'leurs': 55, 'aux': 56, 'monde': 57, 'cette': 58, 'fait': 59, 'entre': 60, 'où': 61, 'faire': 62, 'homme': 63, 'histoire': 64, 'après': 65, 'ont': 66, 'être': 67, 'sans': 68, 'n': 69, 'jour': 70, 'famille': 71, 'comme': 72, 'tous': 73, 'fille': 74, 'père': 75, 'années': 76, 'y': 77, 'amour': 78, 'dont': 79, 'peu': 80, 'temps': 81, 'mère': 82, 'même': 83, 'film': 84, 'ces': 85, 'vont': 86, 'aussi': 87, 'on': 88, 'n’'

In [16]:
print("Texte initial:", X_train.iloc[1])
output = keras_vectorizer([X_train.iloc[1]])
print("Vocabulaire dans le texte (15 premiers items):")
for v in output.numpy()[0, :15]:
    print(v, keras_vectorizer.get_vocabulary()[v])

Texte initial: Leto Leningrad . Un été du début des années 80 . En amont de la Perestroïka , les disques de Lou Reed et de David Bowie s' échangent en contrebande , et une scène rock émerge . Mike et sa femme la belle Natacha rencontrent le jeune Viktor Tsoï . Entourés d’ une nouvelle génération de musiciens , ils vont changer le cours du rock’n’roll en Union Soviétique .
Vocabulaire dans le texte (15 premiers items):
1 [UNK]
1 [UNK]
7 un
94 été
20 du
261 début
11 des
76 années
1060 80
13 en
6887 amont
2 de
3 la
1 [UNK]
8 les


## Chargement de Plongements de Mots Pré-entraînés

In [17]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("../embedding/frWiki_no_phrase_no_postag_700_cbow_cut100.bin", binary=True, unicode_errors="ignore")
model.most_similar("bonjour")

[('merci', 0.7507892847061157),
 ('bonsoir', 0.7450243830680847),
 ('votre', 0.5642200112342834),
 ('vous', 0.5538792014122009),
 ('remercier', 0.5396129488945007),
 ('avance', 0.5288880467414856),
 ('discuter', 0.5033395886421204),
 ('je', 0.49339333176612854),
 ('désoler', 0.4899965822696686),
 ('ici', 0.4887441396713257)]

In [18]:
model.key_to_index

{'</s>': 0,
 'de': 1,
 'la': 2,
 'à': 3,
 'le': 4,
 'et': 5,
 'être': 6,
 'en': 7,
 'les': 8,
 'un': 9,
 'une': 10,
 'avoir': 11,
 'il': 12,
 'dans': 13,
 'par': 14,
 'catégorie': 15,
 'pour': 16,
 'sur': 17,
 'que': 18,
 'se': 19,
 'qui': 20,
 'des': 21,
 'ce': 22,
 'avec': 23,
 'ne': 24,
 'son': 25,
 'an': 26,
 'plus': 27,
 'ou': 28,
 'pas': 29,
 'faire': 30,
 'elle': 31,
 'sa': 32,
 'pouvoir': 33,
 'pop': 34,
 'premier': 35,
 'mais': 36,
 'comme': 37,
 'cette': 38,
 'français': 39,
 'avancement': 40,
 'ses': 41,
 'je': 42,
 'on': 43,
 'deux': 44,
 'france': 45,
 'tout': 46,
 'aussi': 47,
 'lien': 48,
 'grand': 49,
 'année': 50,
 'autre': 51,
 'naissance': 52,
 'article': 53,
 'nom': 54,
 'après': 55,
 'voir': 56,
 'commune': 57,
 'état': 58,
 'ville': 59,
 'externe': 60,
 'y': 61,
 'paris': 62,
 'référence': 63,
 'entre': 64,
 'lui': 65,
 'même': 66,
 'source': 67,
 'sous': 68,
 'jean': 69,
 'lieu': 70,
 'leur': 71,
 'site': 72,
 'note': 73,
 'où': 74,
 'dont': 75,
 'film': 76,
 'ce

In [19]:
model["bonjour"]

array([-4.67579126e-01, -9.34297621e-01, -3.89060646e-01, -2.03962207e-01,
        6.24373436e-01,  5.58501959e-01, -7.94236243e-01,  9.47133780e-01,
       -7.85477459e-01, -4.23343748e-01, -3.78552794e-01,  9.85341787e-01,
       -9.94831264e-01,  7.94630885e-01, -3.72711942e-02, -1.31745422e+00,
       -2.15552300e-01, -1.77847058e-01,  4.44615424e-01,  2.40859807e-01,
       -1.34950832e-01,  2.62641966e-01,  7.27709755e-02, -4.39429581e-01,
        3.13311636e-01,  4.89237309e-01, -3.47837150e-01,  6.98559701e-01,
        3.06835920e-01,  8.79080057e-01, -5.81417084e-01, -3.37294281e-01,
        1.81973651e-01,  7.02408612e-01,  7.81347573e-01,  5.16828001e-01,
        2.21238777e-01, -4.01979834e-01, -3.04546077e-02,  5.94205201e-01,
        7.80208051e-01, -4.02492434e-01,  1.09107220e+00,  1.95819825e-01,
        1.20070234e-01,  4.59087253e-01,  6.24428272e-01,  3.55202556e-02,
       -1.29575109e+00, -5.80421746e-01,  4.97552425e-01,  3.83994550e-01,
       -6.25843883e-01,  

Le plongement pré-entrainé est de dimension 700.

In [20]:
len(model["bonjour"])

700

In [21]:
def load_embeddings(embeddings_model):
    embeddings_index = {}
    for word in list(embeddings_model.key_to_index.keys()):
        embeddings_index[word] = embeddings_model[word]
    print(f'{len(embeddings_index)} vecteurs de mots ont été lus')
    return embeddings_index

In [22]:
embeddings = load_embeddings(model)

39392 vecteurs de mots ont été lus


La cellule suivante permet de créer une matrice de plongements: une matrice où la ligne i correspond au plongement pré-entraîné pour le mot d'indice i dans le vocabulaire.

In [23]:
def get_embedding_matrix(vocabulary, embeddings_index, embedding_dim = 700):
  num_tokens = len(vocabulary)
  hits = 0
  misses = 0

  # Préparation de la matrice
  # Les mots qui ne se trouvent pas dans les plongements pré-entraînés seront 
  # représentés par des vecteurs dont toutes les composantes sont égales à 0,
  # y compris la représentation utilisée pour compléter les documents courts et
  # celle utilisée pour les mots inconnus [UNK]
  embedding_matrix = np.zeros((num_tokens, embedding_dim))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
          hits += 1
      else:
          misses += 1
  print(f'{hits} mots ont été trouvés dans les plongements pré-entraînés')
  print(f'{misses} sont absents')
  return embedding_matrix

In [24]:
# Construction de la matrice de plongements à partir du vocabulaire
embedding_matrix = get_embedding_matrix(voc, embeddings)

4489 mots ont été trouvés dans les plongements pré-entraînés
3511 sont absents


## Construction du CNN

In [25]:
from tensorflow.keras.layers import Embedding

def get_CNN_model(voc_size, embedding_matrix, embedding_dim=700):
  # Création du modèle
  int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
  embedding_layer = Embedding(voc_size, embedding_dim, trainable=True,
      embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
  )
  
  embedded_sequences = embedding_layer(int_sequences_input)
  x = tf.keras.layers.Conv1D(64, 5, activation="relu")(embedded_sequences)
  x = tf.keras.layers.MaxPooling1D(5)(x)
  x = tf.keras.layers.Conv1D(64, 5, activation="relu")(x)
  x = tf.keras.layers.GlobalMaxPooling1D()(x)
  x = tf.keras.layers.Dense(64, activation="relu")(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  preds = tf.keras.layers.Dense(len(genre_name), activation="softmax")(x)
  model = tf.keras.Model(int_sequences_input, preds)
  return model

In [26]:
# Affichage de l'architecture du modèle
CNN_model = get_CNN_model(len(voc), embedding_matrix)
CNN_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 700)         5600000   
                                                                 
 conv1d (Conv1D)             (None, None, 64)          224064    
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 64)          20544     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                              

## Entraînement du CNN

In [27]:
from sklearn import model_selection
from tensorflow.keras.callbacks import EarlyStopping # Early stopping to avoid over-fitting

# Fonction pour l'entraînement d'un modèle
def train_model(X, y, model_function, vectorizer, voc_size, embedding_matrix, embedding_dim=700, batch_size=64):
    
    # Early stopping callback
    early_stopping_callback = EarlyStopping(monitor="val_acc", restore_best_weights=True, patience=3)
    
    # Listes utilisées pour sauvegarder les résultats obtenus à chaque pli
    acc_per_fold = []
    loss_per_fold = []
    histories = []
    folds =  5 # 3
    stratkfold = model_selection.StratifiedKFold(n_splits=folds, shuffle=True, random_state=12)
    fold_no = 1
    for train, test in stratkfold.split(X, y):
        m_function = globals()[model_function]
        model = m_function(voc_size, embedding_matrix, embedding_dim)

        print('------------------------------------------------------------------------')
        print(f'Entraînement pour le pli {fold_no} ...')
        fold_x_train = vectorizer(X.iloc[train].to_numpy()).numpy()
        fold_x_val = vectorizer(X.iloc[test].to_numpy()).numpy()
        fold_y_train = y.iloc[train].to_numpy()
        fold_y_val = y.iloc[test].to_numpy()

        # Compilation du modèle : permet de préciser la fonction de perte et l'optimiseur
        # loss=sparse_categorical_crossentropy : entropie croisée, dans le cas où les 
        # classes cibles sont indiquées sous forme d'entiers. Il s'agira de minimiser
        # la perte pendant l'apprentissage
        # optimizer=rmsprop : l'optimiseur détermine la manière doit les poids seront
        # mis à jour pendant l'apprentissage
        model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
        # Entraînement sur 10 époque (la totalité du jeu de données est parcourue
        # 10 fois)
        history = model.fit(fold_x_train, fold_y_train, batch_size=batch_size, epochs=25, validation_data=(fold_x_val, fold_y_val), callbacks=[early_stopping_callback])
        histories.append(history)
        # Evaluation sur les données de validation
        scores = model.evaluate(fold_x_val, fold_y_val, verbose=0)
        print(f'Scores pour le pli {fold_no}: {model.metrics_names[0]} = {scores[0]:.2f};',
            f'{model.metrics_names[1]} = {scores[1]*100:.2f}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        fold_no = fold_no + 1

    # Affichage des scores moyens par pli
    print('---------------------------------------------------------------------')
    print('Scores par pli')
    for i in range(0, len(acc_per_fold)):
        print('---------------------------------------------------------------------')
        print(f'> Pli {i+1} - Loss: {loss_per_fold[i]:.2f}', f'- Accuracy: {acc_per_fold[i]:.2f}%')
    print('---------------------------------------------------------------------')
    print('Scores moyens pour tous les plis :')
    print(f'> Accuracy: {np.mean(acc_per_fold):.2f}', f'(+- {np.std(acc_per_fold):.2f})')
    print(f'> Loss: {np.mean(loss_per_fold):.2f}')
    print('---------------------------------------------------------------------')
    return histories

In [28]:
# Entraînement du modèle et récupération des résultats
CNN_histories = train_model(X_train, y_train, 'get_CNN_model', keras_vectorizer, len(voc), embedding_matrix)

------------------------------------------------------------------------
Entraînement pour le pli 1 ...
Epoch 1/25
25/57 [============>.................] - ETA: 5s - loss: 2.1882 - acc: 0.1525

KeyboardInterrupt: 

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

def plot_results(histories):
    accuracy_data = []
    loss_data = []
    for i, h in enumerate(histories):
        acc = h.history['acc']
        val_acc = h.history['val_acc']
        loss = h.history['loss']
        val_loss = h.history['val_loss']
        for j in range(len(acc)):
            accuracy_data.append([i+1, j+1, acc[j], 'Entraînement'])
            accuracy_data.append([i+1, j+1, val_acc[j], 'Validation'])
            loss_data.append([i+1, j+1, loss[j], 'Entraînement'])
            loss_data.append([i+1, j+1, val_loss[j], 'Validation'])

    acc_df = pd.DataFrame(accuracy_data, columns=['Pli', 'Epoch', 'Accuracy', 'Données'])
    sns.relplot(data=acc_df, x='Epoch', y='Accuracy', hue='Pli', style='Données', kind='line')
    
    loss_df = pd.DataFrame(loss_data, columns=['Pli', 'Epoch', 'Perte', 'Données'])
    sns.relplot(data=loss_df, x='Epoch', y='Perte', hue='Pli', style='Données', kind='line')

In [ ]:
plot_results(CNN_histories)

## Test du Modèle

In [52]:
# Données d'entrainement
train_data_complete = pd.read_csv("../data/allocine_genres_train.csv", sep=",")
train_data = train_data_complete[["titre", "synopsis", "genre"]]

# Données de test/validation
test_data_complete = pd.read_csv("../data/allocine_genres_test.csv", sep=",")
test_data = test_data_complete[["titre", "synopsis", "genre"]]

In [53]:
X = train_data.drop('genre', axis=1)
y = train_data['genre']

# Initialize the RandomOverSampler
ros = RandomOverSampler()
#ros = RandomUnderSampler()


# Perform oversampling
X_resampled, y_resampled = ros.fit_resample(X, y)

# Convert the resampled data back to a DataFrame
train_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='genre')], axis=1)

In [54]:
# Liste des genres
genre_name = sorted(train_data.genre.unique().flatten())
print("Genres:", genre_name)
print("Nombre d'exemplaires:", len(train_data))

# Identifiant unique par genre
genre2id = {genre_name[i]:i for i in range(len(genre_name))}
id2genre = {i:genre_name[i] for i in range(len(genre_name))}

train_data = train_data.replace({"genre": genre2id})
test_data = test_data.replace({"genre": genre2id})

Genres: ['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction']
Nombre d'exemplaires: 4509


In [55]:
X_train_titre = train_data.titre
X_train = X_train_titre + " " + train_data.synopsis
X_train = keras_vectorizer(X_train)
y_train = train_data.genre

X_test_titre = test_data.titre
X_test = X_test_titre + " " + test_data.synopsis
X_test = keras_vectorizer(X_test)
y_test = test_data.genre

In [45]:
# Compile and train model
early_stopping_callback = EarlyStopping(monitor="val_acc", restore_best_weights=True, patience=3)
CNN_model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
CNN_model.fit(X_train, y_train, batch_size=64, epochs=25, callbacks=[early_stopping_callback])

# Save model
CNN_model.save("model")

Epoch 1/25
71/71 [==============================] - 11s 147ms/step - loss: 2.4947 - acc: 0.1905
Epoch 2/25
71/71 [==============================] - 11s 150ms/step - loss: 1.5596 - acc: 0.4322
Epoch 3/25
71/71 [==============================] - 11s 160ms/step - loss: 0.9572 - acc: 0.6613
Epoch 4/25
71/71 [==============================] - 11s 152ms/step - loss: 0.5349 - acc: 0.8179
Epoch 5/25
71/71 [==============================] - 11s 151ms/step - loss: 0.2966 - acc: 0.9071
Epoch 6/25
71/71 [==============================] - 11s 154ms/step - loss: 0.1600 - acc: 0.9503
Epoch 7/25
71/71 [==============================] - 11s 151ms/step - loss: 0.1278 - acc: 0.9623
Epoch 8/25
71/71 [==============================] - 11s 151ms/step - loss: 0.0868 - acc: 0.9723
Epoch 9/25
71/71 [==============================] - 12s 163ms/step - loss: 0.1036 - acc: 0.9696
Epoch 10/25
71/71 [==============================] - 11s 160ms/step - loss: 0.0697 - acc: 0.9776
Epoch 11/25
71/71 [====================

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [46]:
# Load saved model
CNN_model = tf.keras.models.load_model("model")

In [47]:
results = CNN_model.evaluate(X_test, y_test, batch_size=64)
print("test loss, test acc:", results)

12/12 [==============================] - 1s 53ms/step - loss: 5.7494 - acc: 0.2782
test loss, test acc: [5.749359607696533, 0.2781641185283661]


In [80]:
# Vector of probability for each class
y_pred = CNN_model.predict(X_test)
print(y_pred[0])

# Vector of genre id with the highest probabillity
y_pred = np.argmax(y_pred, axis=1)
print(y_pred[0])

# Convert to dataframe
y_pred = pd.DataFrame(y_pred)

# Rename column and replace genre ids with genre names
y_pred.rename(columns={y_pred.columns[0]: "predicted_genre"}, inplace=True)
y_pred.replace({"predicted_genre": id2genre}, inplace=True)
y_pred

23/23 [==============================] - 1s 32ms/step
[3.6209308e-06 1.5491174e-06 6.5644922e-06 8.4039450e-01 1.5023670e-06
 8.9739455e-04 9.4250701e-08 1.5868081e-01 1.3897812e-05]
3


,predicted_genre
0,drame
1,science fiction
2,science fiction
3,historique
4,science fiction
...,...
714,science fiction
715,romance
716,science fiction
717,horreur


In [81]:
test_data_complete["predicted_genre"] = y_pred
test_data_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                719 non-null    int64  
 1   acteur_1                  719 non-null    object 
 2   acteur_2                  691 non-null    object 
 3   acteur_3                  686 non-null    object 
 4   allocine_id               719 non-null    int64  
 5   annee_prod                719 non-null    int64  
 6   annee_sortie              672 non-null    float64
 7   box_office_fr             438 non-null    float64
 8   couleur                   673 non-null    object 
 9   duree                     706 non-null    float64
 10  langues                   591 non-null    object 
 11  nationalite               719 non-null    object 
 12  nb_critiques_presse       514 non-null    float64
 13  nb_critiques_spectateurs  688 non-null    float64
 14  nb_notes_s

In [67]:
test_data_complete.to_csv("../data/test_results.csv")

In [82]:
print(len(test_data_complete[test_data_complete["genre"] == test_data_complete["predicted_genre"]]))

200


In [85]:
# Vector of probability for each class
y_pred = CNN_model.predict(X_test)
print(y_pred.shape)
print(y_pred[0])

# Vector of genre id with the highest probabillity
y_pred = np.argsort(y_pred, axis=1)
y_pred = np.flip(y_pred, axis=1)
print(y_pred.shape)
print(y_pred[0])

# Convert to dataframe
y_pred = pd.DataFrame(y_pred)

# Rename column and replace genre ids with genre names
y_pred.rename(columns={y_pred.columns[0]: "predicted_genre_1"}, inplace=True)
y_pred.replace({"predicted_genre_1": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[1]: "predicted_genre_2"}, inplace=True)
y_pred.replace({"predicted_genre_2": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[2]: "predicted_genre_3"}, inplace=True)
y_pred.replace({"predicted_genre_3": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[3]: "predicted_genre_4"}, inplace=True)
y_pred.replace({"predicted_genre_4": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[4]: "predicted_genre_5"}, inplace=True)
y_pred.replace({"predicted_genre_5": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[5]: "predicted_genre_6"}, inplace=True)
y_pred.replace({"predicted_genre_6": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[6]: "predicted_genre_7"}, inplace=True)
y_pred.replace({"predicted_genre_7": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[7]: "predicted_genre_8"}, inplace=True)
y_pred.replace({"predicted_genre_8": id2genre}, inplace=True)
y_pred.rename(columns={y_pred.columns[8]: "predicted_genre_9"}, inplace=True)
y_pred.replace({"predicted_genre_9": id2genre}, inplace=True)
y_pred

23/23 [==============================] - 1s 33ms/step
(719, 9)
[3.6209308e-06 1.5491174e-06 6.5644922e-06 8.4039450e-01 1.5023670e-06
 8.9739455e-04 9.4250701e-08 1.5868081e-01 1.3897812e-05]
(719, 9)
[3 7 5 8 2 0 1 4 6]


,predicted_genre_1,predicted_genre_2,predicted_genre_3,predicted_genre_4,predicted_genre_5,predicted_genre_6,predicted_genre_7,predicted_genre_8,predicted_genre_9
0,drame,romance,horreur,science fiction,documentaire,biopic,comédie,historique,policier
1,science fiction,horreur,romance,historique,drame,comédie,policier,biopic,documentaire
2,science fiction,drame,horreur,romance,comédie,historique,policier,documentaire,biopic
3,historique,science fiction,romance,horreur,comédie,biopic,drame,policier,documentaire
4,science fiction,comédie,historique,drame,horreur,romance,policier,biopic,documentaire
...,...,...,...,...,...,...,...,...,...
714,science fiction,horreur,comédie,documentaire,romance,biopic,historique,policier,drame
715,romance,drame,horreur,comédie,biopic,documentaire,science fiction,historique,policier
716,science fiction,horreur,romance,drame,historique,comédie,policier,documentaire,biopic
717,horreur,drame,biopic,policier,documentaire,romance,science fiction,comédie,historique


In [86]:
test_data_complete["predicted_genre_1"] = y_pred["predicted_genre_1"]
test_data_complete["predicted_genre_2"] = y_pred["predicted_genre_2"]
test_data_complete["predicted_genre_3"] = y_pred["predicted_genre_3"]
test_data_complete["predicted_genre_4"] = y_pred["predicted_genre_4"]
test_data_complete["predicted_genre_5"] = y_pred["predicted_genre_5"]
test_data_complete["predicted_genre_6"] = y_pred["predicted_genre_6"]
test_data_complete["predicted_genre_7"] = y_pred["predicted_genre_7"]
test_data_complete["predicted_genre_8"] = y_pred["predicted_genre_8"]
test_data_complete["predicted_genre_9"] = y_pred["predicted_genre_9"]

In [87]:
test_data_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                719 non-null    int64  
 1   acteur_1                  719 non-null    object 
 2   acteur_2                  691 non-null    object 
 3   acteur_3                  686 non-null    object 
 4   allocine_id               719 non-null    int64  
 5   annee_prod                719 non-null    int64  
 6   annee_sortie              672 non-null    float64
 7   box_office_fr             438 non-null    float64
 8   couleur                   673 non-null    object 
 9   duree                     706 non-null    float64
 10  langues                   591 non-null    object 
 11  nationalite               719 non-null    object 
 12  nb_critiques_presse       514 non-null    float64
 13  nb_critiques_spectateurs  688 non-null    float64
 14  nb_notes_s

In [1]:
ppredicted = len(test_data_complete[(test_data_complete["genre"] == test_data_complete["predicted_genre_1"])
                                  | (test_data_complete["genre"] == test_data_complete["predicted_genre_2"])
                                  | (test_data_complete["genre"] == test_data_complete["predicted_genre_3"])])
print("Presque prédit:", ppredicted)
print("Pourcentage:", ppredicted/len(test_data_complete))

NameError: name 'test_data_complete' is not defined